In [1]:
import numpy as np 
import pandas as pd 
import os
import json
import cv2
import matplotlib.pyplot as plt
from skimage import io
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input, ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.utils import to_categorical

IMAGES_FOLDER = "/kaggle/input/glaucoma-datasets/G1020/Images"
df = pd.read_csv("/kaggle/input/glaucoma-datasets/G1020/G1020.csv")

In [2]:
def load_data(df, image_size=(300, 300)):
    images, labels = [], []
    print(f"Total images in dataframe: {len(df)}")

    for index, row in df.iterrows():
        img_name = row['imageID']
        label = row['binaryLabels']
        img_path = os.path.join(IMAGES_FOLDER, img_name)

        if os.path.exists(img_path):
            image = cv2.imread(img_path, cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, image_size)
            images.append(image)
            labels.append(label)
    
    return np.array(images), np.array(labels)

images, labels = load_data(df)
images = images.astype('float32') / 255.0
labels = to_categorical(labels, num_classes=2)

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, stratify=labels, random_state=42)

print(f"Dataset Shape: {images.shape}")
print(f"Train Shape: {X_train.shape}, Test Shape: {X_test.shape}")

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=90,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.1)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow(X_train, y_train, batch_size=8)
test_generator = test_datagen.flow(X_test, y_test, batch_size=8)

HEIGHT = 300
WIDTH = 300
BATCH_SIZE = 8
FC_LAYERS = [1024, 512, 256]
dropout = 0.5
NUM_EPOCHS = 100
NUM_CLASSES = 2  

def build_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        x = Dense(fc, activation='relu')(x)
        x = Dropout(dropout)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    finetune_model = Model(inputs=base_model.input, outputs=predictions)
    return finetune_model

base_model_1 = ResNet50(weights=None, include_top=False, input_shape=(HEIGHT, WIDTH, 3))
base_model_1.load_weights('/kaggle/input/resnet-inception-weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
resnet50_model = build_model(base_model_1, dropout, FC_LAYERS, NUM_CLASSES)

resnet50_model.compile(optimizer=Adam(learning_rate=0.00001), loss="binary_crossentropy", metrics=["accuracy"])

checkpoint = ModelCheckpoint("resnet50_model.keras", monitor="val_accuracy", verbose=1, save_best_only=True, mode="max")
tensorboard = TensorBoard(log_dir="./logs")
callbacks_list = [checkpoint, tensorboard]

resnet50_model.fit(train_generator, epochs=NUM_EPOCHS, validation_data=test_generator, callbacks=[checkpoint, tensorboard])

base_model_2 = InceptionV3(weights= None, include_top=False, input_shape=(HEIGHT, WIDTH, 3))
base_model_2.load_weights('/kaggle/input/resnet-inception-weights/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
inception_model = build_model(base_model_2, dropout, FC_LAYERS, NUM_CLASSES)

inception_model.compile(optimizer=RMSprop(learning_rate=0.00001), loss="binary_crossentropy", metrics=["accuracy"])

history_2 = inception_model.fit(train_generator, epochs=NUM_EPOCHS, validation_data=test_generator)

test_loss, test_acc = resnet50_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

Total images in dataframe: 1020
Dataset Shape: (1020, 300, 300, 3)
Train Shape: (714, 300, 300, 3), Test Shape: (306, 300, 300, 3)
Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


89/90 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step - accuracy: 0.5533 - loss: 1.2179

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: val_accuracy improved from -inf to 0.70915, saving model to resnet50_model.keras
90/90 ━━━━━━━━━━━━━━━━━━━━ 50s 380ms/step - accuracy: 0.5543 - loss: 1.2145 - val_accuracy: 0.7092 - val_loss: 0.6780
Epoch 2/100
89/90 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 0.6212 - loss: 0.8867
Epoch 2: val_accuracy did not improve from 0.70915
90/90 ━━━━━━━━━━━━━━━━━━━━ 15s 150ms/step - accuracy: 0.6213 - loss: 0.8848 - val_accuracy: 0.7092 - val_loss: 0.6131
Epoch 3/100
89/90 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - accuracy: 0.6484 - loss: 0.6892
Epoch 3: val_accuracy did not improve from 0.70915
90/90 ━━━━━━━━━━━━━━━━━━━━ 14s 139ms/step - accuracy: 0.6489 - loss: 0.6886 - val_accuracy: 0.7092 - val_loss: 0.6036
Epoch 4/100
89/90 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.6807 - loss: 0.6636
Epoch 4: val_accuracy did not improve from 0.70915
90/90 ━━━━━━━━━━━━━━━━━━━━ 14s 140ms/step - accuracy: 0.6798 - loss: 0.6640 - val_accuracy: 0.7092 - val_loss: 0.6300
Epoch 5/100
89/90 ━━━━━